<a href="https://colab.research.google.com/github/ViktorSivek/Mistral_voicebot/blob/main/mistral_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Speech to text Whisper

In [2]:
# Step 1: Enable third-party widgets in Google Colab
from google.colab import output
output.enable_custom_widget_manager()

# Step 2: Install
!pip install ipywebrtc
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

# Import necessary classes
from ipywebrtc import AudioRecorder, CameraStream
import re
import subprocess
import io
import sys

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.7/260.7 kB 5.5 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-dv_xi4hq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-dv_xi4hq
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-man

In [3]:
# Step 3: Set up an attempt to record audio (Note: this might not work as expected in Colab)
audio_stream = CameraStream(constraints={'audio': True, 'video': False})
recorder = AudioRecorder(stream=audio_stream)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

In [4]:
# Step 3: Save the recorded audio to a file
from IPython.display import Audio

audio_data = recorder.audio.value
with open('output.mp3', 'wb') as f:
    f.write(audio_data)

In [7]:
# Step 1: Run Whisper and capture the output
# Ensure the model and other parameters are correctly specified
try:
    result = subprocess.run(
        ['whisper', 'output.mp3', '--model', 'medium.en'],
        capture_output=True, text=True, check=True
    )
    whisper_output = result.stdout
except subprocess.CalledProcessError as e:
    print("Failed to transcribe audio:", e)
    whisper_output = ""

# Step 2: Extract only the text from the Whisper output
# Using regex to strip the timestamps and square brackets
if whisper_output:
    clean_text = re.sub(r"^\[.*?\]\s{2}", "", whisper_output, flags=re.MULTILINE)

    # Step 3: Use the cleaned text in a query
    query = "Answer only from provided file. " + clean_text

    # Output the query to see the result
    print(query)
else:
    print("No output from Whisper.")

Answer only from provided file. Hi, my name is Victor. I have so full stomach.



# Mistral RAG


In [1]:
!pip install langchain
!pip install torch
!pip install sentence_transformers
!pip install faiss-cpu
!pip install huggingface-hub
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.0/113.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 7.1 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Usi

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader

In [4]:
#load pdf files
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/data/")
data = loader.load()

In [5]:
print(data)

[Document(page_content=' \n \n', metadata={'source': '/content/drive/MyDrive/data/New-Vital-First-Aid-First-Aid-Book-112019.pdf', 'page': 0}), Document(page_content='Vital First Aid  Ph 1300 88 03 43 \n \n1 \n \t\n \n \nVITAL FIRST AID  \n 2019 \n \nFirst published by Vital First Aid 2012 \nCopyright © Vital First Aid Training Services Pty Ltd,  \n All rights reserved. Without limiting the rights under copyrig ht reserved above, no part of this pub-\nlication may be reproduced, stored in or introduced into a retr ieval system, or tr ansmitted, in any \nform or by any means (electronic, mechanical, photocopying, rec ording or otherwise), without the \nprior written permission of both the copyright owner and the ab ove publisher of this book. \nIllustrated Nicole Humphries \nPrinted in Australia  \n Reprinted November 2019 for electronic distribution, only by Vi tal First Aid Training Services Pty \nLtd, under authority of Vital First Aid  \n  \n \nVital\tFirst\tAid\tTraining \tServices 

In [6]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)

In [7]:
len(text_chunks)

73

In [8]:
#get the third chunk
text_chunks[10]

Document(page_content='Vital First Aid  Ph 1300 88 03 43 \n \n11 \n \nMouth‐to‐nose\tResuscitation \t\n1 Kneel beside the casualty. \n2 Place the casualty on his or her back, with the head tilted ba ck. \n3 Supporting the jaw with your fingers, close the casualty’s mou th and keep it closed, using your \nthumb on the lower lip. \n4 Take a deep breath. Open your mouth wide and place it over the  casualty’s nose, without com-\npressing the soft nostrils. \n5 Breathe into the casualty’s nose. \n6 Move your mouth away. Open the casualty’s lower lip with your thumb to allow exhalation. \n7 Continue as for Mouth-to-mouth Resuscitation. \n\t\nMouth‐to‐mask\tResuscitation \t\n1 Kneel at the top of the casualty’s head, facing the feet.  \n2 Place the narrow end of the mask on the bridge of the casual-\nty’s nose. Use both of your hands, placed on either side of the  jaw, to \nkeep the airway open and to hold the mask firmly in place, crea ting an \nairtight seal. \n3 Take a deep breath and blow

In [9]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [11]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from /content/drive/MyDrive/model/mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   

In [12]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [15]:
# query = "Answer only from provided file. What is CRAMP?"

In [12]:
# Redirect stdout
original_stdout = sys.stdout  # Save a reference to the original standard output
sys.stdout = io.StringIO()

In [ ]:
qa.invoke(query)

In [13]:
# output = """
# Llama.generate: prefix-match hit

# llama_print_timings:        load time =    4235.07 ms
# llama_print_timings:      sample time =      54.77 ms /    92 runs   (    0.60 ms per token,  1679.63 tokens per second)
# llama_print_timings: prompt eval time =  517152.52 ms /  1079 tokens (  479.29 ms per token,     2.09 tokens per second)
# llama_print_timings:        eval time =   63999.74 ms /    91 runs   (  703.29 ms per token,     1.42 tokens per second)
# llama_print_timings:       total time =  571227.74 ms /  1170 tokens
# {'query': 'Answer only from provided file. What is CRAMP?', 'result': ' CRAMP refers to sudden and prolonged muscle contraction causing sharp pain, which may be caused by lack of body fluid, poor circulation or strenuous exercise in extremely hot or cold conditions.'}
# """

# print(output)

In [14]:
# Restore stdout and capture the output
captured_output = sys.stdout.getvalue()  # Retrieve the content written to the StringIO object
sys.stdout = original_stdout

# Text to speech Bark

In [8]:
!pip install git+https://github.com/suno-ai/bark.git

from bark import SAMPLE_RATE, generate_audio, preload_models
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
import json
import re

# for ram problem
# import os
# os.environ["SUNO_OFFLOAD_CPU"] = "True"
# os.environ["SUNO_USE_SMALL_MODELS"] = "True"

# download and load all models
preload_models()

  Cloning https://github.com/suno-ai/bark.git to /tmp/pip-req-build-zx1s7gr1
  Running command git clone --filter=blob:none --quiet https://github.com/suno-ai/bark.git /tmp/pip-req-build-zx1s7gr1
  Resolved https://github.com/suno-ai/bark.git to commit f4f32d4cd480dfec1c245d258174bc9bde3c2148
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 57.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 5.1 MB/s eta 0:00:00
  Created wheel for suno-bark: filename=suno_bark-0.0.1a0-py3-none-any.whl size=2567

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


text_2.pt:   0%|          | 0.00/5.35G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

coarse_2.pt:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

fine_2.pt:   0%|          | 0.00/3.74G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
Downloading: "https://dl.fbaipublicfiles.com/encodec/v0/encodec_24khz-d7cc33bc.th" to /root/.cache/torch/hub/checkpoints/encodec_24khz-d7cc33bc.th
100%|██████████| 88.9M/88.9M [00:00<00:00, 117MB/s]


In [16]:
json_part_match = re.search(r"\{.*\}$", captured_output, re.MULTILINE)
if json_part_match:
    json_part = json_part_match.group(0)

    json_part_corrected = json_part.replace("'", '"')

    # Convert to a Python dictionary
    try:
        result_dict = json.loads(json_part_corrected)

        # Extract the 'result'
        result_text = result_dict['result']

        # Optionally, use or display the extracted result
        print(result_text)
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", e)
else:
    print("No JSON-like part found in the output")

 CRAMP refers to sudden and prolonged muscle contraction causing sharp pain, which may be caused by lack of body fluid, poor circulation or strenuous exercise in extremely hot or cold conditions.


In [9]:
# generate audio from text
text_prompt = f"""
{result_text}
"""

audio_array = generate_audio(text_prompt, history_prompt="v2/en_speaker_1")

# save audio to disk
write_wav("bark_generation.wav", SAMPLE_RATE, audio_array)

# play text in notebook
Audio(audio_array, rate=SAMPLE_RATE)

100%|██████████| 35/35 [14:16<00:00, 24.46s/it]
